# AgentOps
Using [AgentOps](https://agentops.ai/) allows you to quickly and easily add agent observability and monitoring to your Autogen services.

````{=mdx}
:::info Requirements
Some extra dependencies are needed for this notebook, which can be installed via pip:

```bash
pip install pyautogen[agentops]
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

### Adding AgentOps to an existing Autogen service.
It's as simple as adding two lines of code!

In [ ]:
%pip install agentops

In [ ]:
import agentops

agentops.init()

By default, the AgentOps `init()` function will look for an environment variable named `AGENTOPS_API_KEY`. Alternatively, you can pass one in as an optional parameter.

In [ ]:
agentops.init(api_key="xxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxxx")

After initializing AgentOps, Autogen will now start automatically tracking
- LLM calls
- Agent names and actions
- Tool usage
- Correspondence between agents
- Errors
- Token usage and costs

You can retrieve your API key and view the results of your agent runs at [https://agentops.ai](https://agentops.ai)

# Simple Chat Example

In [ ]:
import agentops

from autogen import ConversableAgent, UserProxyAgent, config_list_from_json

agentops.init(api_key="your-api-key-here", tags=["simple-autogen-example"])

config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

# Create the agent that uses the LLM.
assistant = ConversableAgent("agent", llm_config={"config_list": config_list})

# Create the agent that represents the user in the conversation.
user_proxy = UserProxyAgent("user", code_execution_config=False)

# Let the assistant start the conversation.  It will end when the user types exit.
assistant.initiate_chat(user_proxy, message="How can I help you today?")

agentops.end_session("Success")

You can view data on this run at [https://agentops.ai](https://agentops.ai). You should see LLM events for each message sent by each agent, including the user. All data that can be capture is captured.

## Tool Example
In this example lets use Tools with Autogen. You can find more information on this example in [tool-use.ipynb](../website/docs/tutorial/tool-use.ipynb)

In [ ]:
from autogen import ConversableAgent, config_list_from_json, register_function

agentops = None  # removing state from last code block
from typing import Annotated, Literal

import agentops

agentops.init(api_key="your-api-key-here", tags=["autogen-tool-example"])

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")


config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

# Create the agent that uses the LLM.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)
user_proxy.register_for_execution(name="calculator")(calculator)

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

# Let the assistant start the conversation.  It will end when the user types exit.
user_proxy.initiate_chat(assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?")

agentops.end_session("Success")

Again, if you go to [https://agentops.ai](https://agentops.ai), you should see data related to this run including:
- Agents talking to each other
- Each use of the `calculator` tool
- Each call to OpenAI for LLM use